In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
df=pd.read_csv('food_security_final.csv')

In [3]:
df

,Unnamed: 0,caloric_loss,food_prod_var,food_sup_var,gdp_ppp,ader,mder,pop_total,under_perc,under_num,dietary_energy,Year
0,0,2.48,1.4,27.0,2710.3,2204.0,1721.0,1053.1,18.2,264.5,2346.0,2000-01-01
1,1,2.48,1.9,20.0,2792.3,2211.0,1726.0,1071.5,19.8,292.3,2306.0,2001-01-01
2,2,2.44,1.9,23.0,2850.0,2219.0,1731.0,1089.8,20.9,315.0,2272.0,2002-01-01
3,3,2.44,7.3,43.0,3023.6,2227.0,1736.0,1108.0,21.8,333.7,2245.0,2003-01-01
4,4,2.50,7.2,52.0,3210.9,2235.0,1742.0,1126.1,21.9,341.7,2238.0,2004-01-01
5,5,2.51,6.5,49.0,3411.0,2243.0,1747.0,1144.1,21.5,339.8,2253.0,2005-01-01
6,6,2.54,5.9,30.0,3629.4,2250.0,1752.0,1162.0,19.9,320.5,2298.0,2006-01-01
7,7,2.56,4.6,25.0,3848.9,2254.0,1755.0,1179.7,18.4,299.6,2346.0,2007-01-01
8,8,2.63,5.9,49.0,3910.1,2258.0,1758.0,1197.1,17.6,291.0,2369.0,2008-01-01
9,9,2.61,6.9,60.0,4158.4,2263.0,1761.0,1214.3,17.3,290.5,2380.0,2009-01-01


In [4]:
cols=df.columns 
cols=cols.drop('Year','Unnamed: 0')#Since it is currently not related to time series, we remove redundant columns
cols=cols.drop('food_sup_var')#Variance interdependancy is too less, which means predicted values leading up to 2018 will also create bias

In [5]:
from sklearn import preprocessing
mm_scaler = preprocessing.MinMaxScaler()
X_train_minmax = mm_scaler.fit_transform(df[cols]) #Scaling to bring all data to same scaled range, to avoid bias based on large values

In [6]:
X_train_minmax=pd.DataFrame(X_train_minmax)

### Let's Test how Multivariate Regression works for existing data, and understand the linearilty of one key factor to be analysed

In [7]:
from sklearn import linear_model

In [8]:
target=df['under_num'].iloc[0:14]

In [9]:
X=df[cols].iloc[0:14]

In [10]:
lm = linear_model.LinearRegression()
model = lm.fit(X,target)

In [11]:
x=df[cols].iloc[14:17]

In [12]:
predictions = lm.predict(x)
print(predictions)

[293.3 286.1 278.3]


In [13]:
y=df['under_num'].iloc[14:17]

In [14]:
predictions-y

14   -7.958079e-13
15   -9.094947e-13
16   -1.080025e-12
Name: under_num, dtype: float64

In [15]:
from sklearn import metrics

In [16]:
print(np.sqrt(metrics.mean_squared_error(y,predictions))) #The mean square is incredibly low, which means we can continue working with a linear approach

9.357607742205306e-13


### Let's fill in the data for food_prod_var using Multivariate Regression

In [17]:
cols=df.columns
cols=cols.drop('Unnamed: 0','food_prod_var')
cols=cols.drop('Year')

In [18]:
cols=cols.drop('food_prod_var')
cols=cols.drop('food_sup_var')
cols=cols.drop('dietary_energy')

In [19]:
cols

Index(['caloric_loss', 'gdp_ppp', 'ader', 'mder', 'pop_total', 'under_perc',
       'under_num'],
      dtype='object')

In [20]:
X=df[cols].iloc[0:17]
target=df['dietary_energy'].iloc[0:17]

In [21]:
lm = linear_model.LinearRegression()
model = lm.fit(X,target)

In [22]:
x=df[cols].iloc[17:19]
predictions = lm.predict(x)
print(predictions)

[2515.72446974 2512.61729726]


In [23]:
df['dietary_energy'][18]=predictions[1]

/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
df

,Unnamed: 0,caloric_loss,food_prod_var,food_sup_var,gdp_ppp,ader,mder,pop_total,under_perc,under_num,dietary_energy,Year
0,0,2.48,1.4,27.0,2710.3,2204.0,1721.0,1053.1,18.2,264.5,2346.000000,2000-01-01
1,1,2.48,1.9,20.0,2792.3,2211.0,1726.0,1071.5,19.8,292.3,2306.000000,2001-01-01
2,2,2.44,1.9,23.0,2850.0,2219.0,1731.0,1089.8,20.9,315.0,2272.000000,2002-01-01
3,3,2.44,7.3,43.0,3023.6,2227.0,1736.0,1108.0,21.8,333.7,2245.000000,2003-01-01
4,4,2.50,7.2,52.0,3210.9,2235.0,1742.0,1126.1,21.9,341.7,2238.000000,2004-01-01
5,5,2.51,6.5,49.0,3411.0,2243.0,1747.0,1144.1,21.5,339.8,2253.000000,2005-01-01
6,6,2.54,5.9,30.0,3629.4,2250.0,1752.0,1162.0,19.9,320.5,2298.000000,2006-01-01
7,7,2.56,4.6,25.0,3848.9,2254.0,1755.0,1179.7,18.4,299.6,2346.000000,2007-01-01
8,8,2.63,5.9,49.0,3910.1,2258.0,1758.0,1197.1,17.6,291.0,2369.000000,2008-01-01
9,9,2.61,6.9,60.0,4158.4,2263.0,1761.0,1214.3,17.3,290.5,2380.000000,2009-01-01


In [25]:
df=df.drop(['food_prod_var','food_sup_var'],axis=1)

## Lets introduce price indices to this dataset as well. We will work with price indices and some more new data in order to predict the issues that may arise in 2020

In [30]:
price=pd.read_csv('price_index_yearly.csv')

In [31]:
list1=[2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]
price['Year']=pd.to_datetime(list1,format="%Y")

In [32]:
price

,Commodity,PALLFNF,PEXGALL,PFANDB,PFOOD,PBEVE,PAGRI,PRAWM,PFERT,POILAPSP,PCOFFOTM,PMAIZMT,PSUGAISA,Year
0,2000-12-31,60.391199,64.693309,58.101515,59.027975,47.151698,60.056036,73.277852,56.875112,69.501608,85.045756,88.219229,8.079842,2000-01-01
1,2001-12-31,56.191655,60.092256,57.901545,59.601900,37.805057,58.933530,65.914637,54.598995,61.951351,61.909432,89.609138,8.231553,2001-01-01
2,2002-12-31,55.850864,59.359611,61.293249,62.708495,44.566465,62.143663,67.896491,49.792206,62.365504,60.365400,99.333267,6.236949,2002-01-01
3,2003-12-31,65.696901,69.834933,66.908542,68.502411,48.070613,68.364870,78.216537,63.894098,66.923467,64.070603,105.245363,6.925712,2003-01-01
4,2004-12-31,79.175656,84.424369,73.318319,75.373826,49.024293,74.733934,84.310198,74.877678,82.499353,80.097038,111.894866,7.545098,2004-01-01
5,2005-12-31,97.702925,104.724376,75.153943,76.309035,61.501917,76.661681,86.861122,92.234778,117.344852,114.305746,98.507652,10.070184,2005-01-01
6,2006-12-31,114.182640,121.623400,81.328419,82.417950,68.451255,83.606532,99.017365,89.302759,141.649687,113.970791,121.647393,14.788470,2006-01-01
7,2007-12-31,129.096327,137.387581,92.737731,93.919776,78.767152,94.215118,104.209251,129.202204,150.358275,123.253470,163.609891,9.956637,2007-01-01
8,2008-12-31,163.128118,173.683060,109.245391,110.415660,95.413982,109.378849,110.281659,250.282367,197.566223,138.114229,223.360289,12.452207,2008-01-01
9,2009-12-31,116.866220,121.269753,92.698011,92.869151,90.675300,93.135133,96.092149,159.085618,129.466659,141.653218,165.628348,18.150361,2009-01-01


In [33]:
df=df.drop('Unnamed: 0',axis=1)


## Introducing and merging World Bank Statistics

In [34]:
wb=pd.read_csv('world_bank_final.csv')
wb.rename(columns={'Unnamed: 0':'Year'},inplace=True)

In [35]:
 temp=pd.merge(df, wb, how='right', left_on='Year',right_on='Year') 

In [36]:
temp.columns

Index(['caloric_loss', 'gdp_ppp', 'ader', 'mder', 'pop_total', 'under_perc',
       'under_num', 'dietary_energy', 'Year', 'Population, total',
       'Adjusted net national income per capita (annual % growth)',
       'Adjusted net national income (annual % growth)',
       'Adjusted savings: net national savings (% of GNI)',
       'Households and NPISHs Final consumption expenditure (current US$)',
       'Life expectancy at birth, female (years)'],
      dtype='object')

In [37]:
list1=['Year','caloric_loss', 'gdp_ppp', 'ader', 'mder', 'pop_total', 'under_perc',
       'under_num', 'dietary_energy', 'Population, total',
       'Adjusted net national income per capita (annual % growth)',
       'Adjusted net national income (annual % growth)',
       'Adjusted savings: net national savings (% of GNI)',
       'Households and NPISHs Final consumption expenditure (current US$)',
       'Life expectancy at birth, female (years)']

In [38]:
temp=temp.reindex(columns=list1)

In [39]:
temp.corr()>0.6

,caloric_loss,gdp_ppp,ader,mder,pop_total,under_perc,under_num,dietary_energy,"Population, total",Adjusted net national income per capita (annual % growth),Adjusted net national income (annual % growth),Adjusted savings: net national savings (% of GNI),Households and NPISHs Final consumption expenditure (current US$),"Life expectancy at birth, female (years)"
caloric_loss,True,True,True,True,True,False,False,True,True,False,False,False,True,True
gdp_ppp,True,True,True,True,True,False,False,True,True,False,False,False,True,True
ader,True,True,True,True,True,False,False,True,True,False,False,False,True,True
mder,True,True,True,True,True,False,False,True,True,False,False,False,True,True
pop_total,True,True,True,True,True,False,False,True,True,False,False,False,True,True
under_perc,False,False,False,False,False,True,True,False,False,False,False,False,False,False
under_num,False,False,False,False,False,True,True,False,False,False,False,False,False,False
dietary_energy,True,True,True,True,True,False,False,True,True,False,False,False,True,True
"Population, total",True,True,True,True,True,False,False,True,True,False,False,False,True,True
Adjusted net national income per capita (annual % growth),False,False,False,False,False,False,False,False,False,True,True,False,False,False


In [40]:
list1=[2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018]
temp['Year']=pd.to_datetime(list1,format="%Y")

In [41]:
fins=pd.merge(temp, price, how='right', left_on='Year',right_on='Year')

In [42]:
fins.corr()

,caloric_loss,gdp_ppp,ader,mder,pop_total,under_perc,under_num,dietary_energy,"Population, total",Adjusted net national income per capita (annual % growth),...,PFANDB,PFOOD,PBEVE,PAGRI,PRAWM,PFERT,POILAPSP,PCOFFOTM,PMAIZMT,PSUGAISA
caloric_loss,1.000000,0.851572,0.781661,0.782019,0.806425,-0.755271,-0.479889,0.788437,0.803412,0.094600,...,0.575257,0.575680,0.541815,0.563817,0.451146,0.319220,0.455525,0.449611,0.404122,0.371402
gdp_ppp,0.851572,1.000000,0.963133,0.963284,0.978667,-0.883020,-0.520889,0.930349,0.976664,0.312531,...,0.709707,0.704686,0.720791,0.691262,0.525284,0.342731,0.458027,0.602905,0.470952,0.540291
ader,0.781661,0.963133,1.000000,0.999859,0.996201,-0.832624,-0.395183,0.865230,0.996676,0.405092,...,0.831678,0.827442,0.829108,0.815626,0.655774,0.509844,0.611283,0.705790,0.602743,0.653485
mder,0.782019,0.963284,0.999859,1.000000,0.996452,-0.835843,-0.400340,0.868090,0.996928,0.397860,...,0.833767,0.829668,0.829800,0.817247,0.654327,0.513375,0.614745,0.705687,0.605643,0.651762
pop_total,0.806425,0.978667,0.996201,0.996452,1.000000,-0.863615,-0.449450,0.897475,0.999953,0.361285,...,0.820767,0.816270,0.821221,0.804154,0.641586,0.488674,0.590754,0.699890,0.599012,0.647379
under_perc,-0.755271,-0.883020,-0.832624,-0.835843,-0.863615,1.000000,0.836389,-0.989546,-0.862511,-0.077015,...,-0.706122,-0.696856,-0.757477,-0.690310,-0.540987,-0.460036,-0.494475,-0.646361,-0.536049,-0.605818
under_num,-0.479889,-0.520889,-0.395183,-0.400340,-0.449450,0.836389,1.000000,-0.790640,-0.446830,0.253954,...,-0.323504,-0.311750,-0.417941,-0.311854,-0.215991,-0.232176,-0.178428,-0.349119,-0.250736,-0.328225
dietary_energy,0.788437,0.930349,0.865230,0.868090,0.897475,-0.989546,-0.790640,1.000000,0.895514,0.116494,...,0.680655,0.672071,0.726875,0.662164,0.497987,0.386477,0.443087,0.612019,0.487219,0.559746
"Population, total",0.803412,0.976664,0.996676,0.996928,0.999953,-0.862511,-0.446830,0.895514,1.000000,0.363198,...,0.825360,0.820856,0.825637,0.808905,0.646998,0.495258,0.596568,0.704484,0.604654,0.652531
Adjusted net national income per capita (annual % growth),0.094600,0.312531,0.405092,0.397860,0.361285,-0.077015,0.253954,0.116494,0.363198,1.000000,...,0.146909,0.138286,0.220820,0.145528,0.126349,-0.085548,-0.016379,0.184381,-0.064420,0.236357


In [43]:
fins.columns

Index(['Year', 'caloric_loss', 'gdp_ppp', 'ader', 'mder', 'pop_total',
       'under_perc', 'under_num', 'dietary_energy', 'Population, total',
       'Adjusted net national income per capita (annual % growth)',
       'Adjusted net national income (annual % growth)',
       'Adjusted savings: net national savings (% of GNI)',
       'Households and NPISHs Final consumption expenditure (current US$)',
       'Life expectancy at birth, female (years)', 'Commodity', 'PALLFNF',
       'PEXGALL', 'PFANDB', 'PFOOD', 'PBEVE', 'PAGRI', 'PRAWM', 'PFERT',
       'POILAPSP', 'PCOFFOTM', 'PMAIZMT', 'PSUGAISA'],
      dtype='object')

## Bring in inflation and unemployment stats as well

In [44]:
inflate=pd.read_csv('india-inflation-rate-cpi.csv')
unemp=pd.read_csv('india-unemployment-rate.csv')

inflate_temp=inflate[' Inflation Rate (%)'].iloc[40:59]

inflate_temp=pd.DataFrame(inflate_temp)

unemp_temp=unemp[' Unemployment Rate (%)'].iloc[9:28]

inflate_temp['Unemployment']=unemp_temp.values

inflate_temp.index=inflate_temp.index-40

In [45]:
inflate_temp

,Inflation Rate (%),Unemployment
0,4.0094,5.663
1,3.7793,5.659
2,4.2972,5.719
3,3.8059,5.725
4,3.7673,5.669
5,4.2463,5.598
6,5.7965,5.450
7,6.3729,5.323
8,8.3493,5.281
9,10.8824,5.566


In [46]:
list1=[2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018]
inflate_temp['Year']=pd.to_datetime(list1,format="%Y")

In [47]:
pd.merge(fins,inflate_temp,how='left',on='Year')

,Year,caloric_loss,gdp_ppp,ader,mder,pop_total,under_perc,under_num,dietary_energy,"Population, total",...,PBEVE,PAGRI,PRAWM,PFERT,POILAPSP,PCOFFOTM,PMAIZMT,PSUGAISA,Inflation Rate (%),Unemployment
0,2000-01-01,2.48,2710.3,2204.0,1721.0,1053.1,18.2,264.5,2346.000000,1.056576e+09,...,47.151698,60.056036,73.277852,56.875112,69.501608,85.045756,88.219229,8.079842,4.0094,5.663
1,2001-01-01,2.48,2792.3,2211.0,1726.0,1071.5,19.8,292.3,2306.000000,1.075000e+09,...,37.805057,58.933530,65.914637,54.598995,61.951351,61.909432,89.609138,8.231553,3.7793,5.659
2,2002-01-01,2.44,2850.0,2219.0,1731.0,1089.8,20.9,315.0,2272.000000,1.093317e+09,...,44.566465,62.143663,67.896491,49.792206,62.365504,60.365400,99.333267,6.236949,4.2972,5.719
3,2003-01-01,2.44,3023.6,2227.0,1736.0,1108.0,21.8,333.7,2245.000000,1.111523e+09,...,48.070613,68.364870,78.216537,63.894098,66.923467,64.070603,105.245363,6.925712,3.8059,5.725
4,2004-01-01,2.50,3210.9,2235.0,1742.0,1126.1,21.9,341.7,2238.000000,1.129623e+09,...,49.024293,74.733934,84.310198,74.877678,82.499353,80.097038,111.894866,7.545098,3.7673,5.669
5,2005-01-01,2.51,3411.0,2243.0,1747.0,1144.1,21.5,339.8,2253.000000,1.147610e+09,...,61.501917,76.661681,86.861122,92.234778,117.344852,114.305746,98.507652,10.070184,4.2463,5.598
6,2006-01-01,2.54,3629.4,2250.0,1752.0,1162.0,19.9,320.5,2298.000000,1.165486e+09,...,68.451255,83.606532,99.017365,89.302759,141.649687,113.970791,121.647393,14.788470,5.7965,5.450
7,2007-01-01,2.56,3848.9,2254.0,1755.0,1179.7,18.4,299.6,2346.000000,1.183209e+09,...,78.767152,94.215118,104.209251,129.202204,150.358275,123.253470,163.609891,9.956637,6.3729,5.323
8,2008-01-01,2.63,3910.1,2258.0,1758.0,1197.1,17.6,291.0,2369.000000,1.200670e+09,...,95.413982,109.378849,110.281659,250.282367,197.566223,138.114229,223.360289,12.452207,8.3493,5.281
9,2009-01-01,2.61,4158.4,2263.0,1761.0,1214.3,17.3,290.5,2380.000000,1.217726e+09,...,90.675300,93.135133,96.092149,159.085618,129.466659,141.653218,165.628348,18.150361,10.8824,5.566


## To produce results for undernourishment, we have to work with a few parameters that we have analysed before :

=====================================================================================================================

### Particularly: Quantity and Prices of Wheat, Maize, Rice as well as price indices for food, agriculture and raw materials. What might also be included is the net national income and savings, depending on how we analyse their predictions

### We also have to work to compare time series forecasts with actual data that we obtain through multivariate regression, to analyse the true impact on the economy and supply chains

### Try and use fiscal expenditure and gdp contraction for NNI And Savings

In [48]:
rice_price=pd.read_csv('new_india_rice.csv')
wheat_price=pd.read_csv('new_india_wheat.csv')
onion_price=pd.read_csv('new_india_onion.csv')

rice_quantity=pd.read_csv('new_india_rice_quantity.csv')
wheat_quantity=pd.read_csv('new_india_wheat_quantity.csv')
onion_quantity=pd.read_csv('new_india_onion_quantity.csv')

In [49]:
x=[]
x=rice_price['India, Retail, Chennai, Rice, USD/tonne Commodity Code: 1006 ']
x=pd.DataFrame(x)

In [50]:
x=x.rename(columns= {'India, Retail, Chennai, Rice, USD/tonne Commodity Code: 1006 ':'rice_p'})


In [51]:
x['onion_p']=onion_price['India, Retail, New Delhi, Onions, USD/tonne Commodity Code: 71110']

In [52]:
x['wheat_p']=wheat_price['India, Retail, Mumbai, Wheat, USD/tonne Commodity Code: 1001 ']

In [53]:
test=pd.DataFrame()
test['rice_q']=rice_quantity['Value']
test['wheat_q']=wheat_quantity['Value']
test['onion_q']=onion_quantity['Value']

### Now, bringing in Price Indices

In [54]:
x['PFOOD']=fins['PFOOD'].values
x['PAGRI']=fins['PAGRI'].values
x['PRAWM']=fins['PRAWM'].values

In [55]:
from sklearn.preprocessing import StandardScaler
standardScalerX = StandardScaler()
table_final=standardScalerX.fit_transform(x)
table_final= pd.DataFrame(table_final,columns=x.columns)

In [56]:
table_final.head()

,rice_p,onion_p,wheat_p,PFOOD,PAGRI,PRAWM
0,-1.119487,-1.297214,-1.414062,-1.740806,-1.676199,-1.292061
1,-1.309820,-1.222640,-1.533034,-1.712282,-1.731287,-1.629027
2,-1.389637,-1.337370,-1.422560,-1.557882,-1.573748,-1.538330
3,-1.217723,-1.159538,-1.456552,-1.269922,-1.268440,-1.066051
4,-1.187024,-1.039072,-1.397066,-0.928409,-0.955876,-0.787184


### Finally, after scaling, lets make the test set

In [57]:
x_train=table_final.iloc[0:19]
x_test=table_final.iloc[19:]
y=np.log(test)
y=y.iloc[0:19]

In [58]:
lm = linear_model.LinearRegression()
model = lm.fit(x_train,y)

In [59]:
predictions = lm.predict(x_test)
print(predictions)

[[18.94522534 18.44558066 16.78670641]
 [18.95946491 18.61655619 16.54250344]]


In [60]:
np.exp(predictions)

array([[1.68968918e+08, 1.02521251e+08, 1.95152429e+07],
       [1.71392175e+08, 1.21637539e+08, 1.52868485e+07]])

### This seems very close to the actual value , but lets add inflation and unemployment to the data and see the forecast

In [61]:
inflate_temp.loc[19]=[4.54,6.1,2019]

In [62]:
inflate_temp.loc[20]=[6.08,10.661,2020]

In [63]:
list1=[2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]
inflate_temp['Year']=pd.to_datetime(list1,format="%Y")

In [65]:
inflate_temp.to_csv('inflation.csv')

In [66]:
x['Inflation Rate (%)']=inflate_temp[' Inflation Rate (%)']
x['Unemployment']=inflate_temp['Unemployment']

In [67]:
x

,rice_p,onion_p,wheat_p,PFOOD,PAGRI,PRAWM,Inflation Rate (%),Unemployment
0,255.000000,167.500000,230.000000,59.027975,60.056036,73.277852,4.0094,5.663
1,229.166667,178.333333,218.333333,59.601900,58.933530,65.914637,3.7793,5.659
2,218.333333,161.666667,229.166667,62.708495,62.143663,67.896491,4.2972,5.719
3,241.666667,187.500000,225.833333,68.502411,68.364870,78.216537,3.8059,5.725
4,245.833333,205.000000,231.666667,75.373826,74.733934,84.310198,3.7673,5.669
5,260.833333,235.833333,254.166667,76.309035,76.661681,86.861122,4.2463,5.598
6,265.000000,185.833333,304.166667,82.417950,83.606532,99.017365,5.7965,5.450
7,319.166667,369.166667,351.666667,93.919776,94.215118,104.209251,6.3729,5.323
8,383.333333,280.000000,355.000000,110.415660,109.378849,110.281659,8.3493,5.281
9,405.833333,364.166667,349.166667,92.869151,93.135133,96.092149,10.8824,5.566


In [931]:
from sklearn.preprocessing import StandardScaler
standardScalerX = StandardScaler()
table_final=standardScalerX.fit_transform(x)
table_final= pd.DataFrame(table_final,columns=x.columns)

In [932]:
x_train=table_final.iloc[0:19]
x_test=table_final.iloc[19:]
y=np.log(test)
y=y.iloc[0:19]

In [933]:
lm = linear_model.LinearRegression()
model = lm.fit(x_train,y)

In [934]:
predictions = lm.predict(x_test)
print(predictions)

[[18.87029562 18.47099974 16.47245008]
 [18.17980054 18.81082517 13.44107671]]


In [935]:
predictions_add=pd.DataFrame(np.exp(predictions))

In [936]:
predictions_add=predictions_add.rename(columns ={0:'rice_q',1:'wheat_q',2:'onion_q'})

In [937]:
test=test.append(predictions_add)

In [938]:
test.index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
            19, 20]

In [939]:
test

,rice_q,wheat_q,onion_q
0,1.274649e+08,7.636890e+07,4.721100e+06
1,1.399000e+08,6.968090e+07,5.252100e+06
2,1.077303e+08,7.276630e+07,4.209500e+06
3,1.327890e+08,6.576080e+07,6.267600e+06
4,1.246971e+08,7.215620e+07,7.760600e+06
5,1.376901e+08,6.863690e+07,9.432500e+06
6,1.391370e+08,6.935450e+07,1.084700e+07
7,1.445700e+08,7.580670e+07,1.390000e+07
8,1.480360e+08,7.857020e+07,1.356500e+07
9,1.356730e+08,8.067940e+07,1.215880e+07


### Now that we have predicted the production and the variability due to the quantity, let's work with this new data to understand key factors to be impacted due to this 

In [940]:
quantity=test

In [941]:
quantity.head()

,rice_q,wheat_q,onion_q
0,127464896.0,76368896.0,4721100.0
1,139900000.0,69680896.0,5252100.0
2,107730304.0,72766304.0,4209500.0
3,132789000.0,65760800.0,6267600.0
4,124697104.0,72156200.0,7760600.0


In [942]:
x['rice_q']=quantity['rice_q']
x['wheat_q']=quantity['wheat_q']
x['onion_q']=quantity['onion_q']

In [943]:
x

,rice_p,onion_p,wheat_p,PFOOD,PAGRI,PRAWM,Inflation Rate (%),Unemployment,rice_q,wheat_q,onion_q
0,255.000000,167.500000,230.000000,59.027975,60.056036,73.277852,4.0094,5.663,1.274649e+08,7.636890e+07,4.721100e+06
1,229.166667,178.333333,218.333333,59.601900,58.933530,65.914637,3.7793,5.659,1.399000e+08,6.968090e+07,5.252100e+06
2,218.333333,161.666667,229.166667,62.708495,62.143663,67.896491,4.2972,5.719,1.077303e+08,7.276630e+07,4.209500e+06
3,241.666667,187.500000,225.833333,68.502411,68.364870,78.216537,3.8059,5.725,1.327890e+08,6.576080e+07,6.267600e+06
4,245.833333,205.000000,231.666667,75.373826,74.733934,84.310198,3.7673,5.669,1.246971e+08,7.215620e+07,7.760600e+06
5,260.833333,235.833333,254.166667,76.309035,76.661681,86.861122,4.2463,5.598,1.376901e+08,6.863690e+07,9.432500e+06
6,265.000000,185.833333,304.166667,82.417950,83.606532,99.017365,5.7965,5.450,1.391370e+08,6.935450e+07,1.084700e+07
7,319.166667,369.166667,351.666667,93.919776,94.215118,104.209251,6.3729,5.323,1.445700e+08,7.580670e+07,1.390000e+07
8,383.333333,280.000000,355.000000,110.415660,109.378849,110.281659,8.3493,5.281,1.480360e+08,7.857020e+07,1.356500e+07
9,405.833333,364.166667,349.166667,92.869151,93.135133,96.092149,10.8824,5.566,1.356730e+08,8.067940e+07,1.215880e+07


### Let's correlate this again to scale features that may be creating biases in the model

In [944]:
food_sec_pred=x.drop(['PAGRI','PRAWM','rice_p'],axis=1)

In [945]:
food_sec_test=fins[['under_perc']]

In [946]:
from sklearn.preprocessing import StandardScaler
standardScalerX = StandardScaler()
table_final=standardScalerX.fit_transform(food_sec_pred)
table_final= pd.DataFrame(table_final,columns=food_sec_pred.columns)

In [947]:
x_train=table_final.iloc[0:19]
x_test=table_final.iloc[19:]
y=food_sec_test
y=y.iloc[0:19]

In [948]:
lm = linear_model.LinearRegression()
model = lm.fit(x_train,y)

In [949]:
predictions = lm.predict(x_test)
print(predictions)

[[12.69687847]
 [28.79021494]]


In [950]:
new_under_perc=predictions

### Having found the new undernourished percentage , lets work on understanding the population growth for 2019-20

In [951]:
years=fins['pop_total']
years=pd.DataFrame(years)

In [954]:
list1=[2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]
years['Year']=list1

In [955]:
years

,pop_total,Year
0,1053.1,2000
1,1071.5,2001
2,1089.8,2002
3,1108.0,2003
4,1126.1,2004
5,1144.1,2005
6,1162.0,2006
7,1179.7,2007
8,1197.1,2008
9,1214.3,2009


In [956]:
from sklearn.linear_model import LinearRegression
x_x = years.iloc[0:19, 1].values.reshape(-1, 1)
y = years.iloc[0:19, 0].values.reshape(-1, 1)
model = LinearRegression().fit(x_x, y)
y_pred = model.predict([[2019]])
y_pred1=model.predict([[2020]])
y_pred1

array([[1394.03894737]])

In [957]:
years['pop_total'][19]=1377.28947368

/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [958]:
years['pop_total'][20]=1394.03894737

/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [959]:
fins['pop_total']=years['pop_total'].values

In [968]:
fins.columns

Index(['Year', 'caloric_loss', 'gdp_ppp', 'ader', 'mder', 'pop_total',
       'under_perc', 'under_num', 'dietary_energy', 'Population, total',
       'Adjusted net national income per capita (annual % growth)',
       'Adjusted net national income (annual % growth)',
       'Adjusted savings: net national savings (% of GNI)',
       'Households and NPISHs Final consumption expenditure (current US$)',
       'Life expectancy at birth, female (years)', 'PALLFNF', 'PEXGALL',
       'PFANDB', 'PFOOD', 'PBEVE', 'PAGRI', 'PRAWM', 'PFERT', 'POILAPSP',
       'PCOFFOTM', 'PMAIZMT', 'PSUGAISA'],
      dtype='object')

### Having predicted the population of India in 2019-20, lets move on to try to understand income and national savings change for 2020

In [999]:
x

,rice_p,onion_p,wheat_p,PFOOD,PAGRI,PRAWM,Inflation Rate (%),Unemployment,rice_q,wheat_q,onion_q
0,255.000000,167.500000,230.000000,59.027975,60.056036,73.277852,4.0094,5.663,1.274649e+08,7.636890e+07,4.721100e+06
1,229.166667,178.333333,218.333333,59.601900,58.933530,65.914637,3.7793,5.659,1.399000e+08,6.968090e+07,5.252100e+06
2,218.333333,161.666667,229.166667,62.708495,62.143663,67.896491,4.2972,5.719,1.077303e+08,7.276630e+07,4.209500e+06
3,241.666667,187.500000,225.833333,68.502411,68.364870,78.216537,3.8059,5.725,1.327890e+08,6.576080e+07,6.267600e+06
4,245.833333,205.000000,231.666667,75.373826,74.733934,84.310198,3.7673,5.669,1.246971e+08,7.215620e+07,7.760600e+06
5,260.833333,235.833333,254.166667,76.309035,76.661681,86.861122,4.2463,5.598,1.376901e+08,6.863690e+07,9.432500e+06
6,265.000000,185.833333,304.166667,82.417950,83.606532,99.017365,5.7965,5.450,1.391370e+08,6.935450e+07,1.084700e+07
7,319.166667,369.166667,351.666667,93.919776,94.215118,104.209251,6.3729,5.323,1.445700e+08,7.580670e+07,1.390000e+07
8,383.333333,280.000000,355.000000,110.415660,109.378849,110.281659,8.3493,5.281,1.480360e+08,7.857020e+07,1.356500e+07
9,405.833333,364.166667,349.166667,92.869151,93.135133,96.092149,10.8824,5.566,1.356730e+08,8.067940e+07,1.215880e+07


In [985]:
food_sec_ann=x[['PFOOD','Inflation Rate (%)','Unemployment']]

In [986]:
food_sec_ann['pop_total']= fins['pop_total']

/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [987]:
from sklearn.preprocessing import StandardScaler
standardScalerX = StandardScaler()
table_final=standardScalerX.fit_transform(food_sec_ann)
table_final= pd.DataFrame(table_final,columns=food_sec_ann.columns)

In [988]:
x_train=table_final.iloc[0:19]
x_test=table_final.iloc[19:]
y=fins[['Adjusted savings: net national savings (% of GNI)','Adjusted net national income (annual % growth)']]
y=y.iloc[0:19]

In [989]:
lm = linear_model.LinearRegression()
model = lm.fit(x_train,y)

In [990]:
predictions = lm.predict(x_test)
print(predictions)

[[ 14.98798581  10.76110206]
 [-24.58689414  20.78599252]]


In [991]:
new_ann_values=predictions

In [992]:
y=fins[['Households and NPISHs Final consumption expenditure (current US$)']]
y=y.iloc[0:19]

In [993]:
lm = linear_model.LinearRegression()
model = lm.fit(x_train,y)

In [994]:
predictions = lm.predict(x_test)
print(predictions)

[[1.69253154e+12]
 [2.04283864e+12]]


In [996]:
household_pred=predictions

In [997]:
food_sec_ann=x['PFOOD','Inflation Rate (%)','Unemployment','rice_q']
food_sec_ann['pop_total']= fins['pop_total']
food_sec_ann['under_perc']=fins['under']
from sklearn.preprocessing import StandardScaler
standardScalerX = StandardScaler()
table_final=standardScalerX.fit_transform(food_sec_ann)
table_final= pd.DataFrame(table_final,columns=food_sec_ann.columns)

x_train=table_final.iloc[0:19]
x_test=table_final.iloc[19:]
y=fins[['Adjusted savings: net national savings (% of GNI)','Adjusted net national income (annual % growth)']]
y=y.iloc[0:19]

lm = linear_model.LinearRegression()
model = lm.fit(x_train,y)
predictions = lm.predict(x_test)
print(predictions)

[[ 15.53918584  10.91559843]
 [-20.54764134  21.3946052 ]]


/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [1000]:
household_pred

array([[1.69253154e+12],
       [2.04283864e+12]])

In [1003]:
fins

,Year,caloric_loss,gdp_ppp,ader,mder,pop_total,under_perc,under_num,dietary_energy,"Population, total",...,PFANDB,PFOOD,PBEVE,PAGRI,PRAWM,PFERT,POILAPSP,PCOFFOTM,PMAIZMT,PSUGAISA
0,2000-01-01,2.48,2710.3,2204.0,1721.0,1053.100000,18.2,264.5,2346.000000,1.056576e+09,...,58.101515,59.027975,47.151698,60.056036,73.277852,56.875112,69.501608,85.045756,88.219229,8.079842
1,2001-01-01,2.48,2792.3,2211.0,1726.0,1071.500000,19.8,292.3,2306.000000,1.075000e+09,...,57.901545,59.601900,37.805057,58.933530,65.914637,54.598995,61.951351,61.909432,89.609138,8.231553
2,2002-01-01,2.44,2850.0,2219.0,1731.0,1089.800000,20.9,315.0,2272.000000,1.093317e+09,...,61.293249,62.708495,44.566465,62.143663,67.896491,49.792206,62.365504,60.365400,99.333267,6.236949
3,2003-01-01,2.44,3023.6,2227.0,1736.0,1108.000000,21.8,333.7,2245.000000,1.111523e+09,...,66.908542,68.502411,48.070613,68.364870,78.216537,63.894098,66.923467,64.070603,105.245363,6.925712
4,2004-01-01,2.50,3210.9,2235.0,1742.0,1126.100000,21.9,341.7,2238.000000,1.129623e+09,...,73.318319,75.373826,49.024293,74.733934,84.310198,74.877678,82.499353,80.097038,111.894866,7.545098
5,2005-01-01,2.51,3411.0,2243.0,1747.0,1144.100000,21.5,339.8,2253.000000,1.147610e+09,...,75.153943,76.309035,61.501917,76.661681,86.861122,92.234778,117.344852,114.305746,98.507652,10.070184
6,2006-01-01,2.54,3629.4,2250.0,1752.0,1162.000000,19.9,320.5,2298.000000,1.165486e+09,...,81.328419,82.417950,68.451255,83.606532,99.017365,89.302759,141.649687,113.970791,121.647393,14.788470
7,2007-01-01,2.56,3848.9,2254.0,1755.0,1179.700000,18.4,299.6,2346.000000,1.183209e+09,...,92.737731,93.919776,78.767152,94.215118,104.209251,129.202204,150.358275,123.253470,163.609891,9.956637
8,2008-01-01,2.63,3910.1,2258.0,1758.0,1197.100000,17.6,291.0,2369.000000,1.200670e+09,...,109.245391,110.415660,95.413982,109.378849,110.281659,250.282367,197.566223,138.114229,223.360289,12.452207
9,2009-01-01,2.61,4158.4,2263.0,1761.0,1214.300000,17.3,290.5,2380.000000,1.217726e+09,...,92.698011,92.869151,90.675300,93.135133,96.092149,159.085618,129.466659,141.653218,165.628348,18.150361


In [1004]:
new_fins=fins

In [1010]:
new_fins['Households and NPISHs Final consumption expenditure (current US$)'].loc[19]=household_pred[0]

/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [1011]:
new_fins['Households and NPISHs Final consumption expenditure (current US$)'][20]=household_pred[1]

/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [1023]:
new_fins['Adjusted savings: net national savings (% of GNI)'][19]=new_ann_values[0][0]
new_fins['Adjusted savings: net national savings (% of GNI)'][20]=new_ann_values[1][0]
new_fins['Adjusted net national income (annual % growth)'][19]=new_ann_values[0][1]
new_fins['Adjusted net national income (annual % growth)'][20]=new_ann_values[1][1]

/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

In [1026]:
new_fins['under_perc'][19]=new_under_perc[0]
new_fins['under_perc'][20]=new_under_perc[1]

/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [1028]:
new_fins['rice_q']=quantity['rice_q']
new_fins['wheat_q']=quantity['wheat_q']
new_fins['onion_q']=quantity['onion_q']

In [1029]:
new_fins

,Year,caloric_loss,gdp_ppp,ader,mder,pop_total,under_perc,under_num,dietary_energy,"Population, total",...,PAGRI,PRAWM,PFERT,POILAPSP,PCOFFOTM,PMAIZMT,PSUGAISA,rice_q,wheat_q,onion_q
0,2000-01-01,2.48,2710.3,2204.0,1721.0,1053.100000,18.200000,264.5,2346.000000,1.056576e+09,...,60.056036,73.277852,56.875112,69.501608,85.045756,88.219229,8.079842,1.274649e+08,7.636890e+07,4.721100e+06
1,2001-01-01,2.48,2792.3,2211.0,1726.0,1071.500000,19.800000,292.3,2306.000000,1.075000e+09,...,58.933530,65.914637,54.598995,61.951351,61.909432,89.609138,8.231553,1.399000e+08,6.968090e+07,5.252100e+06
2,2002-01-01,2.44,2850.0,2219.0,1731.0,1089.800000,20.900000,315.0,2272.000000,1.093317e+09,...,62.143663,67.896491,49.792206,62.365504,60.365400,99.333267,6.236949,1.077303e+08,7.276630e+07,4.209500e+06
3,2003-01-01,2.44,3023.6,2227.0,1736.0,1108.000000,21.800000,333.7,2245.000000,1.111523e+09,...,68.364870,78.216537,63.894098,66.923467,64.070603,105.245363,6.925712,1.327890e+08,6.576080e+07,6.267600e+06
4,2004-01-01,2.50,3210.9,2235.0,1742.0,1126.100000,21.900000,341.7,2238.000000,1.129623e+09,...,74.733934,84.310198,74.877678,82.499353,80.097038,111.894866,7.545098,1.246971e+08,7.215620e+07,7.760600e+06
5,2005-01-01,2.51,3411.0,2243.0,1747.0,1144.100000,21.500000,339.8,2253.000000,1.147610e+09,...,76.661681,86.861122,92.234778,117.344852,114.305746,98.507652,10.070184,1.376901e+08,6.863690e+07,9.432500e+06
6,2006-01-01,2.54,3629.4,2250.0,1752.0,1162.000000,19.900000,320.5,2298.000000,1.165486e+09,...,83.606532,99.017365,89.302759,141.649687,113.970791,121.647393,14.788470,1.391370e+08,6.935450e+07,1.084700e+07
7,2007-01-01,2.56,3848.9,2254.0,1755.0,1179.700000,18.400000,299.6,2346.000000,1.183209e+09,...,94.215118,104.209251,129.202204,150.358275,123.253470,163.609891,9.956637,1.445700e+08,7.580670e+07,1.390000e+07
8,2008-01-01,2.63,3910.1,2258.0,1758.0,1197.100000,17.600000,291.0,2369.000000,1.200670e+09,...,109.378849,110.281659,250.282367,197.566223,138.114229,223.360289,12.452207,1.480360e+08,7.857020e+07,1.356500e+07
9,2009-01-01,2.61,4158.4,2263.0,1761.0,1214.300000,17.300000,290.5,2380.000000,1.217726e+09,...,93.135133,96.092149,159.085618,129.466659,141.653218,165.628348,18.150361,1.356730e+08,8.067940e+07,1.215880e+07


In [1032]:
food_sec_ann['under_perc']=fins['under_perc']

/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [1033]:
food_sec_ann

,onion_p,PFOOD,Inflation Rate (%),Unemployment,rice_q,pop_total,under_perc
0,167.500000,59.027975,4.0094,5.663,1.274649e+08,1053.100000,18.200000
1,178.333333,59.601900,3.7793,5.659,1.399000e+08,1071.500000,19.800000
2,161.666667,62.708495,4.2972,5.719,1.077303e+08,1089.800000,20.900000
3,187.500000,68.502411,3.8059,5.725,1.327890e+08,1108.000000,21.800000
4,205.000000,75.373826,3.7673,5.669,1.246971e+08,1126.100000,21.900000
5,235.833333,76.309035,4.2463,5.598,1.376901e+08,1144.100000,21.500000
6,185.833333,82.417950,5.7965,5.450,1.391370e+08,1162.000000,19.900000
7,369.166667,93.919776,6.3729,5.323,1.445700e+08,1179.700000,18.400000
8,280.000000,110.415660,8.3493,5.281,1.480360e+08,1197.100000,17.600000
9,364.166667,92.869151,10.8824,5.566,1.356730e+08,1214.300000,17.300000


In [1034]:
from sklearn.preprocessing import StandardScaler
standardScalerX = StandardScaler()
table_final=standardScalerX.fit_transform(food_sec_ann)
table_final= pd.DataFrame(table_final,columns=food_sec_ann.columns)

x_train=table_final.iloc[0:19]
x_test=table_final.iloc[19:]
y=fins[['caloric_loss']]
y=y.iloc[0:19]

lm = linear_model.LinearRegression()
model = lm.fit(x_train,y)
predictions = lm.predict(x_test)
print(predictions)

[[2.8185888 ]
 [1.68853972]]


In [1035]:
caloric_loss_fins=predictions

In [1036]:
new_fins['caloric_loss'][19]=caloric_loss_fins[0]
new_fins['caloric_loss'][20]=caloric_loss_fins[1]

/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [1039]:
x_train=table_final.iloc[0:19]
x_test=table_final.iloc[19:]
y=fins[['dietary_energy']]
y=y.iloc[0:19]

lm = linear_model.LinearRegression()
model = lm.fit(x_train,y)
predictions = lm.predict(x_test)
print(predictions)

[[2599.49969908]
 [2184.9974484 ]]


In [1040]:
dietary_energy_fins=predictions

In [1041]:
new_fins['dietary_energy'][19]=dietary_energy_fins[0]
new_fins['dietary_energy'][20]=dietary_energy_fins[1]

/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [1044]:
food_sec_ann=food_sec_ann.drop(['onion_p','rice_q'],axis=1)

In [1045]:
new_fins.columns

Index(['Year', 'caloric_loss', 'gdp_ppp', 'ader', 'mder', 'pop_total',
       'under_perc', 'under_num', 'dietary_energy', 'Population, total',
       'Adjusted net national income per capita (annual % growth)',
       'Adjusted net national income (annual % growth)',
       'Adjusted savings: net national savings (% of GNI)',
       'Households and NPISHs Final consumption expenditure (current US$)',
       'Life expectancy at birth, female (years)', 'PALLFNF', 'PEXGALL',
       'PFANDB', 'PFOOD', 'PBEVE', 'PAGRI', 'PRAWM', 'PFERT', 'POILAPSP',
       'PCOFFOTM', 'PMAIZMT', 'PSUGAISA', 'rice_q', 'wheat_q', 'onion_q'],
      dtype='object')

In [1047]:
food_sec_ann['Adjusted net national income (annual % growth)']=new_fins['Adjusted net national income (annual % growth)'].values
food_sec_ann['Adjusted savings: net national savings (% of GNI)']=new_fins['Adjusted savings: net national savings (% of GNI)'].values

In [1048]:
food_sec_ann

,PFOOD,Inflation Rate (%),Unemployment,pop_total,under_perc,Adjusted net national income (annual % growth),Adjusted savings: net national savings (% of GNI)
0,59.027975,4.0094,5.663,1053.100000,18.200000,2.794217,16.025157
1,59.601900,3.7793,5.659,1071.500000,19.800000,5.041329,15.751353
2,62.708495,4.2972,5.719,1089.800000,20.900000,4.571101,17.337715
3,68.502411,3.8059,5.725,1108.000000,21.800000,7.622376,19.790915
4,75.373826,3.7673,5.669,1126.100000,21.900000,8.557718,22.972247
5,76.309035,4.2463,5.598,1144.100000,21.500000,8.816796,24.081633
6,82.417950,5.7965,5.450,1162.000000,19.900000,8.322507,26.163648
7,93.919776,6.3729,5.323,1179.700000,18.400000,8.387689,26.753283
8,110.415660,8.3493,5.281,1197.100000,17.600000,2.351215,25.727193
9,92.869151,10.8824,5.566,1214.300000,17.300000,9.589951,24.932586


In [1049]:
from sklearn.preprocessing import StandardScaler
standardScalerX = StandardScaler()
table_final=standardScalerX.fit_transform(food_sec_ann)
table_final= pd.DataFrame(table_final,columns=food_sec_ann.columns)

x_train=table_final.iloc[0:19]
x_test=table_final.iloc[19:]
y=fins[['gdp_ppp']]
y=y.iloc[0:19]

lm = linear_model.LinearRegression()
model = lm.fit(x_train,y)
predictions = lm.predict(x_test)
print(predictions)

[[7103.53205887]
 [6732.1079517 ]]


In [1051]:
gdp_ppp_fins=predictions
new_fins['gdp_ppp'][19]=gdp_ppp_fins[0]
new_fins['gdp_ppp'][20]=gdp_ppp_fins[1]

/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1054]:
new_fins=new_fins.drop('Population, total',axis=1)

In [1057]:
correl=new_fins[['under_perc','pop_total']]
testing=new_fins['under_num']
from sklearn.preprocessing import StandardScaler
standardScalerX = StandardScaler()
table_final=standardScalerX.fit_transform(correl)
table_final= pd.DataFrame(table_final,columns=correl.columns)

x_train=table_final.iloc[0:19]
x_test=table_final.iloc[19:]
y=testing
y=y.iloc[0:19]

lm = linear_model.LinearRegression()
model = lm.fit(x_train,y)
predictions = lm.predict(x_test)
print(predictions)

[254.59478193 519.38449187]


In [1058]:
under_num_new=predictions
new_fins['under_num'][19]=under_num_new[0]
new_fins['under_num'][20]=under_num_new[1]

/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/neelbhandari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1085]:
new_fins.isnull().sum()

Year                                                                 0
caloric_loss                                                         0
gdp_ppp                                                              0
ader                                                                 2
mder                                                                 2
pop_total                                                            0
under_perc                                                           0
under_num                                                            0
dietary_energy                                                       0
Adjusted net national income per capita (annual % growth)            2
Adjusted net national income (annual % growth)                       0
Adjusted savings: net national savings (% of GNI)                    0
Households and NPISHs Final consumption expenditure (current US$)    0
Life expectancy at birth, female (years)                             2
PALLFN

In [1087]:
new_fins=new_fins.drop(['ader','mder','Adjusted net national income per capita (annual % growth)','Life expectancy at birth, female (years)'],axis=1)

In [1089]:
new_fins.to_csv('conclusive_food_sec.csv')